# Resampling observational noise

First we need some imports.

In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import numpy as np
from scipy import stats
import pylab as plt
import torch
import torchist
import swyft

## Training data

Now we generate training data.  As simple example, we consider the model

$$
x = z + \epsilon
$$

where the parameter $z \sim \mathcal{N}(\mu = 0, \sigma = 1)$ is standard normal distributed, and $\epsilon \sim \mathcal{N}(\mu = 0, \sigma = 0.1)$ is a small noise contribution.  We are interested in the posterior of $z$ given a measurement of parameter $x$.

In [27]:
class Simulator(swyft.Simulator):
    def __init__(self):
        super().__init__()
        self.on_after_forward = swyft.to_numpy32

    def forward(self, trace):
        z = trace.sample('z', lambda: np.random.rand(1))
        x = trace.sample('x', lambda z: z + np.random.randn(1)*0.1, z)
        
sim = Simulator()

In [28]:
samples = sim.sample(500)

100%|██████████| 500/500 [00:00<00:00, 22133.76it/s]


In [29]:
resampler = sim.get_resampler(targets = ["x"])

In [30]:
class Network(swyft.SwyftModule):
    def __init__(self):
        super().__init__(lr = 1e-4, lrs_factor = 0.1, early_stopping = False, lrs_patience = 3, early_stopping_patience = 2)
        self.logratios = swyft.LogRatioEstimator_1dim(num_features = 1, num_params = 1, varnames = 'z')

    def forward(self, A, B):
        logratios = self.logratios(A['x'], B['z'])
        return logratios

## Trainer

Training is now done using the `SwyftTrainer` class, which extends `pytorch_lightning.Trainer` by methods like `infer` (see below).

In [31]:
trainer = swyft.SwyftTrainer(accelerator = 'gpu', gpus=1, max_epochs = 20, precision = 32)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


The `swyft.Samples` class provides convenience functions to generate data loaders for training and validation data.

In [32]:
dl_train = samples[:-100].get_dataloader(batch_size = 16, shuffle = True, on_after_load_sample = resampler)
dl_valid = samples[-100:].get_dataloader(batch_size = 16)

In [33]:
network = Network()

In [34]:
trainer.fit(network, dl_train, dl_valid)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                   | Params
-----------------------------------------------------
0 | logratios | LogRatioEstimator_1dim | 17.4 K
-----------------------------------------------------
17.4 K    Trainable params
0         Non-trainable params
17.4 K    Total params
0.070     Total estimated model params size (MB)


Epoch 0:  78%|███████▊  | 25/32 [00:00<00:00, 102.69it/s, loss=-0.0697, v_num=9953689]
Validating: 0it [00:00, ?it/s]
Epoch 1:  78%|███████▊  | 25/32 [00:00<00:00, 105.81it/s, loss=-0.277, v_num=9953689, val_loss=-.00182] 
Validating: 0it [00:00, ?it/s]
Epoch 2:  78%|███████▊  | 25/32 [00:00<00:00, 102.17it/s, loss=-0.43, v_num=9953689, val_loss=-.124] 
Validating: 0it [00:00, ?it/s]
Epoch 3:  78%|███████▊  | 25/32 [00:00<00:00, 106.38it/s, loss=-0.54, v_num=9953689, val_loss=-.399] 
Validating: 0it [00:00, ?it/s]
Epoch 4:  78%|███████▊  | 25/32 [00:00<00:00, 116.98it/s, loss=-0.578, v_num=9953689, val_loss=-.481]
Validating: 0it [00:00, ?it/s]
Epoch 5:  81%|████████▏ | 26/32 [00:00<00:00, 109.90it/s, loss=-0.553, v_num=9953689, val_loss=-.493]
Validating: 0it [00:00, ?it/s]
Epoch 6:  81%|████████▏ | 26/32 [00:00<00:00, 104.07it/s, loss=-0.605, v_num=9953689, val_loss=-.480]
Validating: 0it [00:00, ?it/s]
Epoch 7:  81%|████████▏ | 26/32 [00:00<00:00, 107.02it/s, loss=-0.562, v_num=9953